<a href="https://colab.research.google.com/github/rwcitek/anl-nerde/blob/rwc-02/nerde_argonne_api_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NERDE API via Python



NERDE ( National Economic Resilience Data Explorer )



- https://www.anl.gov/dis/national-economic-resilience-data-explorer-nerde
- https://nerde.anl.gov/
- [Introduction to Nerde]( https://youtu.be/FCzSLtNCG5U?feature=shared ) on YouTube
- [API Docs]( https://nerde.anl.gov/resources/api-documentation )
  - [account registration/sign in]( https://nerde.anl.gov/auth/login )


Other resources:
- [Federal Information Processing Standards (FIPS) ]( https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards )

## Setting up the Request


### Obtaining an API Token


In [1]:
# Get u/p credentials from Colab Vault
from google.colab import userdata

nerde_username = userdata.get('nerde_username')
nerde_password = userdata.get('nerde_password')


In [2]:
import requests
import json
import pandas as pd


In [3]:
call = "https://nerde.anl.gov"

# Pull API Token
response = requests.post(
  url=f"{call}/api/auth/login",
  json={
    "userName": nerde_username,  # Remember to replace your username
    "password": nerde_password,  # Remember to replace your password
    "rememberMe": True
  }
)
response


<Response [200]>

In [4]:
token = response.json()

key = token['result']['token'] # Your unique key
f"{key[:20]}...{key[-20:]}"

'eyJhbGciOiJIUzI1NiIs...o-WSN2E6FiI6spas3hx0'

### Establishing parameters




In [5]:
# FIPS codes
my_fips = '24011,24013,24015,24017,24019,24021,24023,24025,24027,24029,24031,24033,24035,24039,24037,24041,24043,24045,24047'
# Start and End dates
my_start = '2020-01-01'
my_end = '2023-01-01'


In [6]:
len(my_fips.split(","))

19

### Defining the function to make the data request




In [7]:
def NERDE_API(section_url, fips="24011", startdate='2020-01-01', enddate='2023-01-01'):
  response = requests.get(
    url=f"{call}{section_url}?county_fips={fips}&start_date={startdate}&end_date={enddate}",
    headers={"Authorization": f"Bearer {key}"}
  ).json()

  # Each county will populate a list, this loops through each called county to parse results
  return [ result for result in response['result'] ]


## Getting County Level Data



### Summary Data



In [8]:
# Create the variable to point to the correct dataset you'd like to pull data from
section_url = "/api/v1/county/explorer/summary"
section_url


'/api/v1/county/explorer/summary'

In [9]:
# API call
summary_digest = NERDE_API(section_url, my_fips, my_start, my_end)


In [10]:
len(summary_digest)

19

In [11]:
# Create a Pandas DataFrame from summary_digest
main_summary_df = pd.DataFrame(summary_digest)


In [12]:
main_summary_df.shape

(19, 14)

In [13]:
main_summary_df[:5]

,county_fips,county,state,total_population,census_per_capita_income,designated_coal_community,adjacent_tribal_land,nuclear_power_plant_present,persistent_poverty_designation,pct_urban_2020,pct_rural_2020,bea_per_capita_income,local_24month_unemployment_rate_series,edci_indicators
0,24011,Caroline County,Maryland,33406,33898,NaN,None,None,No,0.150452,0.849548,56447,"[{'value': 0.0372016509117358, 'timestamp': '2...","[{'score': -1.69499694620526, 'capacity': 'Inf..."
1,24013,Carroll County,Maryland,174318,51465,NaN,None,None,No,0.577612,0.422388,74895,"[{'value': 0.0296015539496292, 'timestamp': '2...","[{'score': -1.63513699632672, 'capacity': 'Inf..."
2,24015,Cecil County,Maryland,104366,42560,NaN,None,None,No,0.516520,0.483480,59373,"[{'value': 0.0397263205400315, 'timestamp': '2...","[{'score': -1.80146427534243, 'capacity': 'Inf..."
3,24017,Charles County,Maryland,168710,50540,1.0,None,None,No,0.715851,0.284149,66642,"[{'value': 0.0352469640229852, 'timestamp': '2...","[{'score': -3, 'capacity': 'Infrastructure', '..."
4,24019,Dorchester County,Maryland,32612,34780,NaN,None,None,No,0.460422,0.539578,57060,"[{'value': 0.0424337032877077, 'timestamp': '2...","[{'score': -1.23962080337455, 'capacity': 'Hum..."


In [14]:
main_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   county_fips                             19 non-null     object 
 1   county                                  19 non-null     object 
 2   state                                   19 non-null     object 
 3   total_population                        19 non-null     int64  
 4   census_per_capita_income                19 non-null     int64  
 5   designated_coal_community               7 non-null      float64
 6   adjacent_tribal_land                    0 non-null      object 
 7   nuclear_power_plant_present             0 non-null      object 
 8   persistent_poverty_designation          19 non-null     object 
 9   pct_urban_2020                          19 non-null     float64
 10  pct_rural_2020                          19 non-null     float64


In [15]:
main_summary_df["designated_coal_community"].value_counts( dropna=False)

,count
designated_coal_community,
NaN,12
1.0,7


In [16]:
main_summary_df["nuclear_power_plant_present"].value_counts( dropna=False)

,count
nuclear_power_plant_present,
None,19


Our returned dataframe contains binary column data that has been returned as a floating number as well as several missing values. Let's clean that up...



### Cleaning the Dataframe



In [17]:
def update_column_values(df, column_name):
  # Replace 1.0 with 'Yes' and NaN with 'No'
  df[column_name] = df[column_name].apply(lambda x: 'Yes' if x == 1.0 else ( 'No' if pd.isna(x) else x ))

  # Set column to string type
  df[column_name] = df[column_name].astype(str)

  return df


In [18]:
main_summary_df = update_column_values(main_summary_df, 'designated_coal_community')
main_summary_df = update_column_values(main_summary_df, 'nuclear_power_plant_present')


In [19]:
main_summary_df["designated_coal_community"].value_counts( dropna=False)

,count
designated_coal_community,
No,12
Yes,7


In [20]:
main_summary_df["nuclear_power_plant_present"].value_counts( dropna=False)

,count
nuclear_power_plant_present,
No,19


### Separate Unemploment data and EDCI Indicators and flattened nested dictionaries




In [21]:
# Create function to un-nest columns
def flatten_column_data(df, list_column):
  # explode list_column and convert dictionaries to fields
  df_1 = df.explode(list_column)[list_column].apply(pd.Series)

  # join data frames on their index and drop original list column
  return df.join(df_1).drop(columns = list_column)


In [22]:
sorted(main_summary_df.columns)

['adjacent_tribal_land',
 'bea_per_capita_income',
 'census_per_capita_income',
 'county',
 'county_fips',
 'designated_coal_community',
 'edci_indicators',
 'local_24month_unemployment_rate_series',
 'nuclear_power_plant_present',
 'pct_rural_2020',
 'pct_urban_2020',
 'persistent_poverty_designation',
 'state',
 'total_population']

In [23]:
# Unemployment Data
unemployment_edci_data = main_summary_df[['county', 'county_fips', 'local_24month_unemployment_rate_series']]


In [24]:
unemployment_edci_data.shape

(19, 3)

In [25]:
unemployment_edci_data[:5]

,county,county_fips,local_24month_unemployment_rate_series
0,Caroline County,24011,"[{'value': 0.0372016509117358, 'timestamp': '2..."
1,Carroll County,24013,"[{'value': 0.0296015539496292, 'timestamp': '2..."
2,Cecil County,24015,"[{'value': 0.0397263205400315, 'timestamp': '2..."
3,Charles County,24017,"[{'value': 0.0352469640229852, 'timestamp': '2..."
4,Dorchester County,24019,"[{'value': 0.0424337032877077, 'timestamp': '2..."


In [26]:
# Create a new unemployment DataFrame with flattened data
flattened_unemployment_data = flatten_column_data(unemployment_edci_data, 'local_24month_unemployment_rate_series')


In [27]:
flattened_unemployment_data.shape

(703, 4)

In [28]:
flattened_unemployment_data[:5]

,county,county_fips,value,timestamp
0,Caroline County,24011,0.037202,2020-02-01
0,Caroline County,24011,0.037232,2020-03-01
0,Caroline County,24011,0.037641,2020-01-01
0,Caroline County,24011,0.037756,2023-01-01
0,Caroline County,24011,0.039085,2020-04-01


In [29]:
# EDCI Indicators Data
edci_data = main_summary_df[['county', 'county_fips', 'edci_indicators']]


In [30]:
# Create a new EDCI DataFrame with flattened data
flattened_edci_data = flatten_column_data(edci_data, 'edci_indicators')


In [31]:
flattened_edci_data.shape

(1121, 7)

In [32]:
flattened_edci_data[:5]

,county,county_fips,score,capacity,indicator,percentile,directionality
0,Caroline County,24011,-1.694997,Infrastructure,perc_selfsupply,0.075941,negative
0,Caroline County,24011,-1.211996,Infrastructure,perc_stream_impaired,0.156863,negative
0,Caroline County,24011,-1.130224,Institutions and Partnerships,local_govt_employees_per_capita,0.032991,positive
0,Caroline County,24011,-0.798065,Industry Composition,estabs_exit_rate,0.203859,negative
0,Caroline County,24011,-0.636447,Human Capital,pct_gross_rent_as_pct_of_hhold_income_above_35pct,0.245876,negative


> **Tip**
>
> If you run into `KeyError: 'result'` the API Key has expired and you should re-run the section [Obtaining an API Token](#obtaining-an-api-token)



### Demographics



In [33]:
# Demographics API end point
demographics_url="/api/v1/county/explorer/demographics"
demographics_url


'/api/v1/county/explorer/demographics'

In [34]:
# Calling the API
demographics_digest = NERDE_API(demographics_url, my_fips, my_start, my_end)


In [35]:
len(demographics_digest)

19

In [36]:
# Turn response into Dataframe
demographics_df = pd.DataFrame(demographics_digest)


In [37]:
demographics_df.shape

(19, 27)

In [38]:
demographics_df[:5]

,county_fips,county,state,total_population,county_persons_in_poverty,county_pct_with_health_insurance,estimate_hispanic_or_latino_any_race,estimate_non_hispanic_or_latino,estimate_non_hispanic_black_or_african_american_alone,estimate_non_hispanic_american_indian_and_alaskan_native_alone,...,educ_attainment_associates_degree,educ_attainment_bachelors_degree,educ_attainment_graduate_professional,language_spoken_at_home_speak_only_english,language_spoken_at_home_speak_spanish,language_spoken_at_home_speak_spanish_english_very_well,language_spoken_at_home_speak_spanish_english_less_very_well,pct_county_pop_urban,pct_county_pop_rural,annual_demographics
0,24011,"Caroline County, Maryland",Maryland,33406,0.124,0.929,2930,30476,4424,10,...,1508,2323,1690,28613,2051,701,1350,0.150452,0.849548,"[{'year': 2020, 'male_median_age': 37.7, 'tota..."
1,24013,"Carroll County, Maryland",Maryland,174318,0.053,0.972,8333,165985,6772,146,...,10487,29204,19058,153978,4624,3164,1460,0.577612,0.422388,"[{'year': 2020, 'male_median_age': 40.4, 'tota..."
2,24015,"Cecil County, Maryland",Maryland,104366,0.109,0.962,5617,98749,7167,92,...,5875,11635,7976,92133,3116,2139,977,0.516520,0.483480,"[{'year': 2020, 'male_median_age': 40, 'total_..."
3,24017,"Charles County, Maryland",Maryland,168710,0.066,0.956,12648,156062,81750,465,...,10268,20308,15820,142444,7683,5491,2192,0.715851,0.284149,"[{'year': 2020, 'male_median_age': 36.4, 'tota..."
4,24019,"Dorchester County, Maryland",Maryland,32612,0.163,0.946,1909,30703,8254,0,...,1814,2521,1891,29148,1210,629,581,0.460422,0.539578,"[{'year': 2020, 'male_median_age': 44.2, 'tota..."


Let's clean up that county field.

In [39]:
demographics_df["county_old"] = demographics_df["county"]
demographics_df[:5]

,county_fips,county,state,total_population,county_persons_in_poverty,county_pct_with_health_insurance,estimate_hispanic_or_latino_any_race,estimate_non_hispanic_or_latino,estimate_non_hispanic_black_or_african_american_alone,estimate_non_hispanic_american_indian_and_alaskan_native_alone,...,educ_attainment_bachelors_degree,educ_attainment_graduate_professional,language_spoken_at_home_speak_only_english,language_spoken_at_home_speak_spanish,language_spoken_at_home_speak_spanish_english_very_well,language_spoken_at_home_speak_spanish_english_less_very_well,pct_county_pop_urban,pct_county_pop_rural,annual_demographics,county_old
0,24011,"Caroline County, Maryland",Maryland,33406,0.124,0.929,2930,30476,4424,10,...,2323,1690,28613,2051,701,1350,0.150452,0.849548,"[{'year': 2020, 'male_median_age': 37.7, 'tota...","Caroline County, Maryland"
1,24013,"Carroll County, Maryland",Maryland,174318,0.053,0.972,8333,165985,6772,146,...,29204,19058,153978,4624,3164,1460,0.577612,0.422388,"[{'year': 2020, 'male_median_age': 40.4, 'tota...","Carroll County, Maryland"
2,24015,"Cecil County, Maryland",Maryland,104366,0.109,0.962,5617,98749,7167,92,...,11635,7976,92133,3116,2139,977,0.516520,0.483480,"[{'year': 2020, 'male_median_age': 40, 'total_...","Cecil County, Maryland"
3,24017,"Charles County, Maryland",Maryland,168710,0.066,0.956,12648,156062,81750,465,...,20308,15820,142444,7683,5491,2192,0.715851,0.284149,"[{'year': 2020, 'male_median_age': 36.4, 'tota...","Charles County, Maryland"
4,24019,"Dorchester County, Maryland",Maryland,32612,0.163,0.946,1909,30703,8254,0,...,2521,1891,29148,1210,629,581,0.460422,0.539578,"[{'year': 2020, 'male_median_age': 44.2, 'tota...","Dorchester County, Maryland"


In [40]:
demographics_df["county"] = demographics_df["county_old"].str.partition(",")[0]
demographics_df[:5]

,county_fips,county,state,total_population,county_persons_in_poverty,county_pct_with_health_insurance,estimate_hispanic_or_latino_any_race,estimate_non_hispanic_or_latino,estimate_non_hispanic_black_or_african_american_alone,estimate_non_hispanic_american_indian_and_alaskan_native_alone,...,educ_attainment_bachelors_degree,educ_attainment_graduate_professional,language_spoken_at_home_speak_only_english,language_spoken_at_home_speak_spanish,language_spoken_at_home_speak_spanish_english_very_well,language_spoken_at_home_speak_spanish_english_less_very_well,pct_county_pop_urban,pct_county_pop_rural,annual_demographics,county_old
0,24011,Caroline County,Maryland,33406,0.124,0.929,2930,30476,4424,10,...,2323,1690,28613,2051,701,1350,0.150452,0.849548,"[{'year': 2020, 'male_median_age': 37.7, 'tota...","Caroline County, Maryland"
1,24013,Carroll County,Maryland,174318,0.053,0.972,8333,165985,6772,146,...,29204,19058,153978,4624,3164,1460,0.577612,0.422388,"[{'year': 2020, 'male_median_age': 40.4, 'tota...","Carroll County, Maryland"
2,24015,Cecil County,Maryland,104366,0.109,0.962,5617,98749,7167,92,...,11635,7976,92133,3116,2139,977,0.516520,0.483480,"[{'year': 2020, 'male_median_age': 40, 'total_...","Cecil County, Maryland"
3,24017,Charles County,Maryland,168710,0.066,0.956,12648,156062,81750,465,...,20308,15820,142444,7683,5491,2192,0.715851,0.284149,"[{'year': 2020, 'male_median_age': 36.4, 'tota...","Charles County, Maryland"
4,24019,Dorchester County,Maryland,32612,0.163,0.946,1909,30703,8254,0,...,2521,1891,29148,1210,629,581,0.460422,0.539578,"[{'year': 2020, 'male_median_age': 44.2, 'tota...","Dorchester County, Maryland"


In [41]:
# Annual Demographics Dataframe
demographics_nested_df = demographics_df[['county', 'county_fips', 'annual_demographics']]


In [42]:
demographics_nested_df.shape

(19, 3)

In [43]:
demographics_nested_df[:5]

,county,county_fips,annual_demographics
0,Caroline County,24011,"[{'year': 2020, 'male_median_age': 37.7, 'tota..."
1,Carroll County,24013,"[{'year': 2020, 'male_median_age': 40.4, 'tota..."
2,Cecil County,24015,"[{'year': 2020, 'male_median_age': 40, 'total_..."
3,Charles County,24017,"[{'year': 2020, 'male_median_age': 36.4, 'tota..."
4,Dorchester County,24019,"[{'year': 2020, 'male_median_age': 44.2, 'tota..."


In [44]:
# Annual Demographics Un-nested
demographics_annnual_df = flatten_column_data(demographics_nested_df, 'annual_demographics')


In [45]:
demographics_annnual_df.shape

(76, 8)

In [46]:
demographics_annnual_df[:5]

,county,county_fips,year,male_median_age,total_median_age,total_population,female_median_age,median_household_income
0,Caroline County,24011,2020.0,37.7,39.3,33260.0,40.7,59042.0
0,Caroline County,24011,2021.0,38.0,39.2,33234.0,40.0,63027.0
0,Caroline County,24011,2022.0,38.4,39.5,33320.0,40.3,65326.0
0,Caroline County,24011,2023.0,38.7,39.8,33406.0,40.7,66368.0
1,Carroll County,24013,2020.0,40.4,42.2,168233.0,43.8,99569.0


### Housing


In [47]:
#Housing API end point
housing_url="/api/v1/county/explorer/housing"
housing_url

'/api/v1/county/explorer/housing'

In [48]:
# Calling the API
housing_digest = NERDE_API(housing_url, my_fips, my_start, my_end)


In [49]:
len(housing_digest)

19

In [50]:
housing_digest[:1]

[{'county_fips': '24011',
  'county': 'Caroline County',
  'state': 'Maryland',
  'ookla_median_download_speed_mbps': 55.8219986,
  'ookla_median_upload_speed_mbps': 8.616000175,
  'mean_iai_value': 0.292128566317267,
  'median_month_owner_costs': 1299,
  'median_property_value': 258800,
  'median_real_estate_taxes_paid': 2518,
  'pct_pop_gross_mortgage_pct_income_below_20pct': 36,
  'pct_pop_gross_mortgage_pct_income_20_24_9pct': 17,
  'pct_pop_gross_mortgage_pct_income_25_29_9pct': 12,
  'pct_pop_gross_mortgage_pct_income_30_34_9pct': 10,
  'pct_pop_gross_mortgage_pct_income_above_35pct': 23,
  'pct_pop_gross_rent_pct_income_below_15pct': 13,
  'pct_pop_gross_rent_pct_income_15_19_9pct': 10,
  'pct_pop_gross_rent_pct_income_20_24_9pct': 10,
  'pct_pop_gross_rent_pct_income_25_29_9pct': 11,
  'pct_pop_gross_rent_pct_income_30_34_9pct': 14,
  'pct_pop_gross_rent_pct_income_above_35pct': 40,
  'means_of_transportation_drove_alone_car_truck_or_van': 12855,
  'means_of_transportation_carp

In [51]:
# Turn response into Dataframe
housing_df = pd.DataFrame(housing_digest)


In [52]:
housing_df.shape

(19, 29)

In [53]:
housing_df[:5]

,county_fips,county,state,ookla_median_download_speed_mbps,ookla_median_upload_speed_mbps,mean_iai_value,median_month_owner_costs,median_property_value,median_real_estate_taxes_paid,pct_pop_gross_mortgage_pct_income_below_20pct,...,pct_pop_gross_rent_pct_income_above_35pct,means_of_transportation_drove_alone_car_truck_or_van,means_of_transportation_carpool_car_truck_or_van,means_of_transportation_public_transportation,means_of_transportation_walked,means_of_transportation_taxicab_motorcycle_bicycle_or_other,means_of_transportation_worked_from_home,means_of_transportation_worked_in_county_of_residence,means_of_transportation_worked_outside_county_of_residence,means_of_transportation_worked_outside_state_of_residence
0,24011,Caroline County,Maryland,55.821999,8.616,0.292129,1299,258800,2518,36,...,40,12855,1114,57,432,467,844,6350,7810,1609
1,24013,Carroll County,Maryland,108.137001,11.574,0.232711,1865,390200,3751,49,...,38,70085,5770,453,882,503,12341,45222,40934,3878
2,24015,Cecil County,Maryland,85.745003,11.627,0.313327,1537,292500,2983,44,...,42,38294,5108,314,825,606,4237,25647,10281,13456
3,24017,Charles County,Maryland,105.861999,38.159,0.353366,2123,382800,4071,42,...,39,64265,5882,3597,538,753,9777,37258,22440,25114
4,24019,Dorchester County,Maryland,25.535000,6.095,0.232348,992,226000,2263,40,...,46,11806,1530,52,159,133,1001,8666,5220,795


### Workforce



In [54]:
# Workforce API end point
workforce_url = "/api/v1/county/explorer/workforce"
workforce_url

'/api/v1/county/explorer/workforce'

In [55]:
# Calling the API
workforce_digest = NERDE_API(workforce_url, my_fips, my_start, my_end)


In [56]:
len(workforce_digest)

19

In [57]:
# Turn response into Dataframe
workforce_df = pd.DataFrame(workforce_digest)


In [58]:
workforce_df.shape

(19, 7)

In [59]:
workforce_df[:5]

,county_fips,county,state,prime_age_population,prime_age_employment_level,prime_age_employment_rate,monthly_economic_stats
0,24011,Caroline County,Maryland,12204,9428,0.772534,"[{'timestamp': '2020-01-01', 'county_labor_for..."
1,24013,Carroll County,Maryland,63841,54386,0.851898,"[{'timestamp': '2020-01-01', 'county_labor_for..."
2,24015,Cecil County,Maryland,39661,30714,0.774413,"[{'timestamp': '2020-01-01', 'county_labor_for..."
3,24017,Charles County,Maryland,66424,56024,0.843430,"[{'timestamp': '2020-01-01', 'county_labor_for..."
4,24019,Dorchester County,Maryland,11131,8530,0.766328,"[{'timestamp': '2020-01-01', 'county_labor_for..."


In [60]:
# Monthly Economic Statistics Dataframe
workforce_monthly_stats = workforce_df[['county', 'county_fips', 'monthly_economic_stats']]


In [61]:
workforce_monthly_stats.shape

(19, 3)

In [62]:
workforce_monthly_stats[:5]

,county,county_fips,monthly_economic_stats
0,Caroline County,24011,"[{'timestamp': '2020-01-01', 'county_labor_for..."
1,Carroll County,24013,"[{'timestamp': '2020-01-01', 'county_labor_for..."
2,Cecil County,24015,"[{'timestamp': '2020-01-01', 'county_labor_for..."
3,Charles County,24017,"[{'timestamp': '2020-01-01', 'county_labor_for..."
4,Dorchester County,24019,"[{'timestamp': '2020-01-01', 'county_labor_for..."


In [63]:
# Monthly Economic Statistics Un-nested
workforce_monthly_stats_flattened = flatten_column_data(workforce_monthly_stats, 'monthly_economic_stats')


In [64]:
workforce_monthly_stats_flattened.shape


(703, 6)

In [65]:
workforce_monthly_stats_flattened[:5]

,county,county_fips,timestamp,county_labor_force,county_unemployment_rate,county_24month_unemployment_rate
0,Caroline County,24011,2020-01-01,0.519092,0.040,0.037641
0,Caroline County,24011,2020-02-01,0.517980,0.040,0.037202
0,Caroline County,24011,2020-03-01,0.518280,0.046,0.037232
0,Caroline County,24011,2020-04-01,0.490649,0.081,0.039085
0,Caroline County,24011,2020-05-01,0.490710,0.078,0.040762


### Local Economy



In [66]:
# Local Economy URL
localeconomy_url = "/api/v1/county/explorer/localeconomy"
localeconomy_url

'/api/v1/county/explorer/localeconomy'

In [67]:
# Calling the API
localeconomy_digest = NERDE_API(localeconomy_url, my_fips, my_start, my_end)


In [68]:
len(localeconomy_digest)

19

In [69]:
# Turn response into Dataframe
localeconomy_df = pd.DataFrame(localeconomy_digest)


In [70]:
localeconomy_df.shape

(19, 5)

In [71]:
localeconomy_df[:5]

,county_fips,county,state,industry_employment_series,industry_gdp_series
0,24011,Caroline County,Maryland,"[{'year': '2020', 'industry': 'Accommodation a...","[{'year': '2020', 'industry': 'Accommodation a..."
1,24013,Carroll County,Maryland,"[{'year': '2020', 'industry': 'Accommodation a...","[{'year': '2020', 'industry': 'Accommodation a..."
2,24015,Cecil County,Maryland,"[{'year': '2020', 'industry': 'Accommodation a...","[{'year': '2020', 'industry': 'Accommodation a..."
3,24017,Charles County,Maryland,"[{'year': '2020', 'industry': 'Accommodation a...","[{'year': '2020', 'industry': 'Accommodation a..."
4,24019,Dorchester County,Maryland,"[{'year': '2020', 'industry': 'Accommodation a...","[{'year': '2020', 'industry': 'Accommodation a..."


In [72]:
# Industry Employment Dataframe
localeconomy_industry_employment_series = localeconomy_df[['county', 'county_fips', 'state', 'industry_employment_series']]


In [73]:
localeconomy_industry_employment_series.shape

(19, 4)

In [74]:
localeconomy_industry_employment_series[:5]

,county,county_fips,state,industry_employment_series
0,Caroline County,24011,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
1,Carroll County,24013,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
2,Cecil County,24015,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
3,Charles County,24017,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
4,Dorchester County,24019,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."


In [75]:
# Industry Employment Un-nested
localeconomy_industry_employment_series_flattened = flatten_column_data(localeconomy_industry_employment_series, 'industry_employment_series')


In [76]:
localeconomy_industry_employment_series_flattened.shape

(1197, 6)

In [77]:
localeconomy_industry_employment_series_flattened[:5]

,county,county_fips,state,year,industry,employment_value
0,Caroline County,24011,Maryland,2020,Accommodation and food services,507
0,Caroline County,24011,Maryland,2020,Administrative and support and waste managemen...,742
0,Caroline County,24011,Maryland,2020,"Arts, entertainment, and recreation",183
0,Caroline County,24011,Maryland,2020,Construction,1593
0,Caroline County,24011,Maryland,2020,Educational services,0


In [78]:
# Industry GDP Dataframe
localeconomy_industry_gdp_series = localeconomy_df[['county', 'county_fips', 'state', 'industry_gdp_series']]


In [79]:
localeconomy_industry_gdp_series.shape

(19, 4)

In [80]:
localeconomy_industry_gdp_series[:5]

,county,county_fips,state,industry_gdp_series
0,Caroline County,24011,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
1,Carroll County,24013,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
2,Cecil County,24015,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
3,Charles County,24017,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."
4,Dorchester County,24019,Maryland,"[{'year': '2020', 'industry': 'Accommodation a..."


In [81]:
# Industry GDP Un-nested
localeconomy_industry_gdp_series_flattened = flatten_column_data(localeconomy_industry_gdp_series, 'industry_gdp_series')


In [82]:
localeconomy_industry_gdp_series_flattened.shape

(1596, 6)

In [83]:
localeconomy_industry_gdp_series_flattened[:5]

,county,county_fips,state,year,industry,gdp_value
0,Caroline County,24011,Maryland,2020,Accommodation and food services,14403
0,Caroline County,24011,Maryland,2020,Administrative and support and waste managemen...,28488
0,Caroline County,24011,Maryland,2020,"Agriculture, forestry, fishing and hunting",3315
0,Caroline County,24011,Maryland,2020,"Arts, entertainment, and recreation",2438
0,Caroline County,24011,Maryland,2020,Construction,124069


### Industry Trends



In [84]:
# Industry Trends URL
industrytrends_url="/api/v1/county/explorer/industrytrends"
industrytrends_url

'/api/v1/county/explorer/industrytrends'

In [85]:
# Calling the API
industry_digest = NERDE_API(industrytrends_url, my_fips, my_start, my_end)


In [86]:
len(industry_digest)

76

In [87]:
# Turn response into Dataframe
industry_df = pd.DataFrame(industry_digest)


In [88]:
industry_df.shape

(76, 10)

In [89]:
industry_df[:5]

,county_fips,county,state,number_of_patent_issuances_all_time,number_of_patent_issuances_last_year,total_sbir_grants,total_sbir_amount,total_sttr_grants,total_sttr_amount,location_quotient_series
0,24011,Caroline County,Maryland,NaN,NaN,NaN,NaN,NaN,NaN,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
1,24011,Caroline County,Maryland,NaN,NaN,NaN,NaN,NaN,NaN,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
2,24011,Caroline County,Maryland,NaN,NaN,NaN,NaN,NaN,NaN,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
3,24011,Caroline County,Maryland,NaN,NaN,NaN,NaN,NaN,NaN,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
4,24013,Carroll County,Maryland,1.0,1.0,13.0,9060895.74,2.0,574441.0,"[{'rank': 1, 'year': 2023, 'value': 2.07, 'sta..."


In [90]:
# Industry Employment Dataframe
industry_location_quotient_series_df = industry_df[['county', 'county_fips', 'location_quotient_series']]


In [91]:
industry_location_quotient_series_df.shape

(76, 3)

In [92]:
industry_location_quotient_series_df[:5]

,county,county_fips,location_quotient_series
0,Caroline County,24011,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
1,Caroline County,24011,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
2,Caroline County,24011,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
3,Caroline County,24011,"[{'rank': 1, 'year': 2023, 'value': 4.87, 'sta..."
4,Carroll County,24013,"[{'rank': 1, 'year': 2023, 'value': 2.07, 'sta..."


In [93]:
# Industry Employment Un-nested
industry_location_quotient_series_df_flattened = flatten_column_data(industry_location_quotient_series_df, 'location_quotient_series')


In [94]:
industry_location_quotient_series_df_flattened.shape

(3892, 9)

In [95]:
industry_location_quotient_series_df_flattened[:5]

,county,county_fips,rank,year,value,status,attribute,growth_rate,naics_title_plot
0,Caroline County,24011,1,2023,4.87,neg,avg_estabs,-0.141667,"Agriculture, forestry, fishing and hunting (11)"
0,Caroline County,24011,1,2023,5.53,neg,avg_employment,-0.921667,"Agriculture, forestry, fishing and hunting (11)"
0,Caroline County,24011,1,2023,6.80,neg,annual_wages,-1.133333,"Agriculture, forestry, fishing and hunting (11)"
0,Caroline County,24011,2,2023,2.12,neg,avg_employment,-0.078333,Construction (23)
0,Caroline County,24011,2,2023,2.24,stable,avg_estabs,0.005000,Construction (23)


### Risk And Resilience


In [96]:
# Rist and Resilience URL
risk_url="/api/v1/county/explorer/riskresilience"
risk_url

'/api/v1/county/explorer/riskresilience'

In [97]:
# Calling the API
risk_digest = NERDE_API(risk_url, my_fips, my_start, my_end)


In [98]:
len(risk_digest)

2

In [99]:
# Turn response into Dataframe
risk_df = pd.DataFrame(risk_digest)


In [100]:
risk_df.shape

(2, 67)

In [101]:
risk_df[:5]

,geo_fips,county,state,risk_score,risk_rating,risk_state_percentile,expected_annual_loss_score,expected_annual_loss_rating,expected_annual_loss_total,avalanche_expected_annual_loss_total,...,climrr_tempmin_seas_mid85_hist_autumn_aggregate_value,climrr_tempminann_mid85_hist_aggregate_value,climrr_heatindex_c_m85_dmax_aggregate_value,climrr_heatindex_c_m85_smax_aggregate_value,climrr_heatindex_c_m85_d95_aggregate_value,climrr_heatindex_c_m85_d105_aggregate_value,climrr_heatindex_c_m85_d115_aggregate_value,climrr_heatindex_c_m85_d125_aggregate_value,gini_index,fema_disaster_declarations
0,24019,Dorchester County,Maryland,75.469297,Relatively Low,37.500000,74.713711,Relatively Low,1.377875e+07,None,...,3.508887,3.403084,6.395606,16.595637,14.560922,3.875345,2.040023,1.543916,0.4656,"[{'incident_type': 'Hurricane', 'disaster_numb..."
1,24037,St. Mary's County,Maryland,81.959911,Relatively Low,66.666667,82.822656,Relatively Low,2.168395e+07,None,...,3.529878,3.325836,6.650624,17.573277,16.732652,4.776655,2.300933,1.651493,0.4069,"[{'incident_type': 'Hurricane', 'disaster_numb..."


In [102]:
# Rename Column
risk_df.rename(columns={"geo_fips": "county_fips"}, inplace=True)


In [103]:
[ x for x in risk_df.columns if "fips" in x.lower() ]


['county_fips']

In [104]:
# Risk FEMA Disaster Declarations Dataframe
risk_fema_disaster_dec_df = risk_df[['county', 'county_fips', 'fema_disaster_declarations']]


In [105]:
risk_fema_disaster_dec_df.shape

(2, 3)

In [106]:
risk_fema_disaster_dec_df[:5]

,county,county_fips,fema_disaster_declarations
0,Dorchester County,24019,"[{'incident_type': 'Hurricane', 'disaster_numb..."
1,St. Mary's County,24037,"[{'incident_type': 'Hurricane', 'disaster_numb..."


In [107]:
# Risk FEMA Disaster Declarations Un-nested
risk_fema_disaster_dec_df_flatten = flatten_column_data(risk_fema_disaster_dec_df, 'fema_disaster_declarations')


In [108]:
risk_fema_disaster_dec_df_flatten.shape

(2, 8)

In [109]:
risk_fema_disaster_dec_df_flatten[:5]

,county,county_fips,incident_type,disaster_number,declaration_date,declaration_type,declaration_title,average_disasters_per_year_county
0,Dorchester County,24019,Hurricane,4583,2021-02-04,DR,TROPICAL STORM ISAIAS,1
1,St. Mary's County,24037,Hurricane,4583,2021-02-04,DR,TROPICAL STORM ISAIAS,1
